In [1]:
import os
import numpy as np
import pickle

# Batch

In [2]:
version = '220829_dFBA_batch_v06'
loc     = f'/mnt/itching_scratch/mgotsmy/220829_slim_paper_v2/results/{version}'
q_pDNA_min = .1/100
special_interest_q_pDNA_max_levels = np.array([1,1.5,2,4])*q_pDNA_min
all_q_pDNA_max_levels = np.linspace(.1/100,.5/100,41)
# check if special_interest_q_pDNA_max_levels in all_q_pDNA_max_levels
for q in special_interest_q_pDNA_max_levels:
    if np.any(np.isclose(q,all_q_pDNA_max_levels)):
        pass
    else:
        print('q_pDNA_max = {:6.4f} mmol/(g h) from special interest not in all.'.format(q))

In [3]:
# lookup stored files
filenames                  = sorted(os.listdir(loc))
q_pDNA_max_levels_set      = all_q_pDNA_max_levels
files_extra                = []
all_filenames              = []
special_interest_filenames = []

# check if there are extra files or files missing
for filename in filenames:
    q_pDNA_max = float(filename.strip('.pkl').split('_')[-1])
    isclose = np.isclose(q_pDNA_max,q_pDNA_max_levels_set)
    if np.any(isclose):
        q_pDNA_max_levels_set = q_pDNA_max_levels_set[np.invert(isclose)]
        all_filenames.append(filename)
        if np.any(np.isclose(q_pDNA_max,special_interest_q_pDNA_max_levels)):
            special_interest_filenames.append(filename)
    else:
        files_extra.append([filename,q_pDNA_max])

# print results
print('Reading files.')
if len(q_pDNA_max_levels_set):
    print('Files not found:')
    for q_pDNA_max in sorted(q_pDNA_max_levels_set):
        print('{:30} {:6.4f}'.format('',q_pDNA_max))
else:
    print('All files found.')
if len(files_extra):
    print('Extra files found:')
    for filename, q_pDNA_max in files_extra:
        print('{:30} {:6.4f}'.format(filename,q_pDNA_max))
else:
    print('No extra files found.')

Reading files.
All files found.
No extra files found.


In [4]:
def batch_results_sorter(results):
    '''
    Sorts results dic according to numeric value in key.
    '''
    results_sorted = {}
    for S_0 in sorted(results):
        results_sorted[S_0] = results[S_0]
    return results_sorted

def batch_unpack_values(sol):
    '''
    Unpacks concentrations in the sol.y array.
    '''
    t = sol.t
    X = sol.y.T[:,0]
    S = sol.y.T[:,1]
    P = sol.y.T[:,2]
    G = sol.y.T[:,3]
    return t,X,S,P,G

def batch_calculate_terminal_idx(sol):
    '''
    Calculates terminal_idx and start of starvation time.
    '''
    
    t,X,S,P,G = batch_unpack_values(sol)
    
    # calculate average termination time. either when infeasible or when G < 0.
    sorted_t_events_ATPM = sorted(sol.t_events[0])
    sorted_t_events_G    = sorted(sol.t_events[1])
    if len(sorted_t_events_ATPM) > 0:
        terminal_idx = np.argmin(
            np.abs(sol.t-
                   np.mean(
                   np.array(sorted_t_events_ATPM)[
                   np.where(
                   np.isclose(sorted_t_events_ATPM[0],sorted_t_events_ATPM,atol=.1))[0]])))
    elif len(sorted_t_events_G) > 0:
        terminal_idx = np.argmin(np.abs(sol.t-np.min(sorted_t_events_G)))
    else:
        terminal_idx = -1

    # find t_starvation_start
    s_is_0 = np.isclose(S,0,atol=1e-4)
    if np.any(s_is_0):
        idx_star_start = np.where(s_is_0)[0][0]          # index of t_starvation_start
        t_star_start   = t[idx_star_start]               # value of t_starvation_start
    else:
        t_star_start = np.nan
    
    return terminal_idx, t_star_start

def batch_calculate_endpoints(results):
    '''
    Return concentrations of molecules of interest at the end of a process.
    '''
    
    endpoints = []
    for S_0 in sorted(results):
        sol = results[S_0]
        t,X,S,P,G = batch_unpack_values(sol)
        terminal_idx, t_star_start = batch_calculate_terminal_idx(sol)
        endpoint = [
            S_0,
            t[terminal_idx],
            X[terminal_idx],
            S[terminal_idx],
            P[terminal_idx],
            G[terminal_idx],
            t_star_start
        ]
        endpoints.append(endpoint)
    return np.array(endpoints)

def batch_trim_processes(results):
    '''
    Trim processes of special interest to the calculated process ends.
    '''
    
    trimmed_results = {}
    for S_0 in sorted(results):
        sol = results[S_0]
        t,X,S,P,G = batch_unpack_values(sol)
        terminal_idx, t_star_start = batch_calculate_terminal_idx(sol)
        curves = [
            S_0,
            t[:terminal_idx],
            X[:terminal_idx],
            S[:terminal_idx],
            P[:terminal_idx],
            G[:terminal_idx],
            t_star_start
        ]
        trimmed_results[S_0] = curves
    return trimmed_results

In [5]:
print('Processing files.')
processed = {}
special_interest = {}
for filename in all_filenames:
    q_pDNA_max = float(filename.strip('.pkl').split('_')[-1])
    with open(f'{loc}/{filename}','rb') as file:
        results = batch_results_sorter(pickle.load(file))
    processed[q_pDNA_max] = batch_calculate_endpoints(results)
    if filename in special_interest_filenames:
        special_interest[q_pDNA_max] = batch_trim_processes(results)
    # break
print('Done.')
print('Length processed        = {:3.0f}'.format(len(processed)))
print('Length special interest = {:3.0f}'.format(len(special_interest)))

Processing files.
Done.
Length processed        =  41
Length special interest =   4


In [6]:
write_loc = f'{version}.pkl'
with open(write_loc,'wb') as file:
    pickle.dump([processed,special_interest],file)
print('Saved processed file.')

Saved processed file.
